In [2]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
from datetime import datetime

## Generate Random Cities

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

729

## Retrieve weather data

In [6]:
# Import config from another folder
import sys
sys.path.append('../')
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [7]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [13]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_desc = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_desc,
                          "Date": city_date})
    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

cord 29 of Set 6 | ranong
Processing Record 30 of Set 6 | jacqueville
Processing Record 31 of Set 6 | cherskiy
Processing Record 32 of Set 6 | ipswich
Processing Record 33 of Set 6 | kununurra
Processing Record 34 of Set 6 | kenai
Processing Record 35 of Set 6 | angoche
Processing Record 36 of Set 6 | yazoo city
Processing Record 37 of Set 6 | zverinogolovskoye
Processing Record 38 of Set 6 | vallenar
Processing Record 39 of Set 6 | derzhavinsk
Processing Record 40 of Set 6 | bubaque
Processing Record 41 of Set 6 | teyateyaneng
Processing Record 42 of Set 6 | da nang
Processing Record 43 of Set 6 | barbar
City not found. Skipping...
Processing Record 44 of Set 6 | chumikan
Processing Record 45 of Set 6 | kotovo
Processing Record 46 of Set 6 | torrington
Processing Record 47 of Set 6 | alanya
Processing Record 48 of Set 6 | la ligua
Processing Record 49 of Set 6 | nome
Processing Record 50 of Set 6 | houma
Processing Record 1 of Set 7 | mogadishu
Processing Record 2 of Set 7 | bargal
Ci

In [14]:
len(city_data)

664

In [15]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,Maceio,BR,-9.6658,-35.7353,82.40,65,75,11.50,broken clouds,2021-03-25 19:13:48
1,Clyde River,CA,70.4692,-68.5914,-4.00,84,75,12.66,snow,2021-03-25 19:14:20
2,Klaksvik,FO,62.2266,-6.5890,44.60,87,90,12.66,overcast clouds,2021-03-25 19:06:08
3,Chuy,UY,-33.6971,-53.4616,74.14,81,100,11.48,moderate rain,2021-03-25 19:14:20
4,Busselton,AU,-33.6500,115.3333,51.01,93,25,8.08,scattered clouds,2021-03-25 19:14:20
5,Thompson,CA,55.7435,-97.8558,6.80,66,90,6.91,overcast clouds,2021-03-25 19:14:20
6,Albany,US,42.6001,-73.9662,70.00,72,49,1.99,scattered clouds,2021-03-25 19:15:50
7,Port Alfred,ZA,-33.5906,26.8910,70.47,83,62,15.95,broken clouds,2021-03-25 19:15:51
8,Ushuaia,AR,-54.8000,-68.3000,53.60,54,40,25.32,light rain,2021-03-25 19:15:51
9,Medzev,SK,48.7004,20.8937,43.00,61,40,5.75,scattered clouds,2021-03-25 19:14:21


In [16]:
# Export the City_Data into a CSV.
city_data_df.to_csv("WeatherPy_Database.csv", index_label="City_ID")